# ZHCPA Dingtalk

Resources:
- [Dingtalk Dev Docs](https://open.dingtalk.com/document/orgapp/create-an-approval-form-template?spm=ding_open_doc.document.0.0.53244a97oo0wyS)
- [Dingtalk Backend API Explorer](https://open-dev.dingtalk.com/apiExplorer#/?devType=org&api=workflow_1.0%23ListProcessInstanceIds)
- [Dingtalk Frontend API Explorer](https://open-dev.dingtalk.com/apiExplorer#/jsapi?api=device.notification.extendModal)
- [Dingtalk Admin Dashboard](https://oa.dingtalk.com/admin/portal/oa#?lang=zh_CN&nation=HK&code=b885d25ac75b3ad2a203ab9fcb3d0799)

## Database Schema

opuser
- id
- code (zhcpa internal thing)
- name
- role
    - Partner	
    - Principal	
    - Senior Manager	
    - Manager	
    - Assistant Manager	
    - Senior 2	
    - Senior 1	
    - Junior 2	
    - Junior 1	
    - Internship
    - Comsec	
    - Admin
- email

attendance
- id
- opuser_id
- is_on_duty (check_type)
- time (user_check_time)
- is_missed (location_result)
    - ~~Normal：范围内~~
    - ~~Outside：范围外~~
    - NotSigned：未打卡
- source
    - ATM：考勤机打卡（指纹/人脸打卡）
    - BEACON：IBeacon
    - DING_ATM：钉钉考勤机（考勤机蓝牙打卡）
    - USER：用户打卡
    - BOSS：老板改签
    - APPROVE：审批系统
    - SYSTEM：考勤系统
    - AUTO_CHECK：自动打卡
- schedule_type (group_id) (i.e. HK, MY, CN)
- ~~time_result~~
    - Normal：正常
    - Early：早退
    - Late：迟到
    - SeriousLate：严重迟到
    - Absenteeism：旷工迟到
    - NotSigned：未打卡

leave
- id (process_instance_id)
- user_id (originator_user_id)
- application_create_time (create_time)
- application_
- is_approved (result)
- status

overtime

## Work/Leave Calculations

Attendance 考勤 + (OT + Leave) OA

### Attendance

- Late or not late depends on the schedule of the op user. 

### OT

- Need to subtract `abort` from `success` applications (by ID?)
- Maybe a new workflow to cater for the `Food money amount` column?
- Maybe a new workflow to cater for the `Job code` + `Duration`? For `Job code` only extract the first 4 characters. Need to summarise how much time each user "OT" on each job
- ~~Only need to concern with `Duration`~~

### Leave

- Need to subtract `abort` from `success` applications (by ID?)
- ~~Only need to concern with `Duration`~~ 

### Flow

For each user:

1. Get all attendance

## Get vacation types

In [6]:
from alibabacloud_dingtalk.attendance_1_0 import models as dingtalk_attendance_models

def get_vacation_types() -> List[dingtalk_attendance_models.AddLeaveTypeResponseBodyResult]:
    try:
        data = api(
            "https://oapi.dingtalk.com/topapi/attendance/vacation/type/list",
            {
                "op_userid": admin_opuserid,
                "vacation_source": "all"
            },
        )
        return data

    except Exception as err:
        raise err

## Get operation users leave records

In [7]:
from alibabacloud_dingtalk.attendance_1_0 import models as dingtalk_attendance_models
from alibabacloud_dingtalk.attendance_1_0.client import (
    Client as DingtalkAttendanceClient,
)


def get_opusers_leave_records(
    leave_code: str, opuserids: List[str]
) -> List[dingtalk_attendance_models.GetLeaveRecordsResponseBodyResultLeaveRecords]:
    def fetch_from_server(
        pageOffset: int,
    ) -> Tuple[
        List[dingtalk_attendance_models.GetLeaveRecordsResponseBodyResultLeaveRecords],
        Optional[int],
    ]:
        req = dingtalk_attendance_models.GetLeaveRecordsRequest(
            op_user_id=admin_opuserid,
            user_ids=opuserids,
            leave_code=leave_code,
            page_number=pageOffset,
            page_size=200,
        )
        headers = dingtalk_attendance_models.GetLeaveRecordsHeaders(
            x_acs_dingtalk_access_token=access_token
        )

        try:
            response = DingtalkAttendanceClient(config).get_leave_records_with_options(
                req, headers, runtime=runtime_options
            )
            if getattr(response, "status_code") != 200:
                raise Exception("Request not ok")
            
            if not getattr(response.body, "success"):
                raise Exception(f"Dingtalk request not ok")
            
            data = response.body.result

            return (
                data.leave_records,
                pageOffset + 1 if getattr(data, "has_more") else None,
            )
        except Exception as err:
            raise err

    return generate_depagination_logic(fetch_from_server)()

In [12]:
vacation_types_df = pd.DataFrame(vacation_types)
vacation_types_df

,freedom_leave,hours_in_per_day,leave_code,leave_hour_ceil,leave_name,leave_time_ceil_min_unit,leave_view_unit,natural_day_leave,paid_leave,source,when_can_leave
0,True,800,f60377e7-9c60-48bb-91e2-5d02f4415361,up,年假(小時),halfHour,hour,false,False,inner,entry
1,True,800,c4d0316f-c234-4d89-8c66-c90b5931d072,,年假(天),NaN,day,false,False,inner,entry
2,True,800,c02aa600-8dce-4b63-9945-35d346464ef6,down,病假,halfHour,hour,false,False,inner,entry
3,True,800,a9047b4b-0d7f-4d7c-a57d-6cb89f7d3b91,,考試假,hour,day,false,False,inner,entry
4,True,800,52ed3423-0612-41c4-ace8-dc6b0c098030,down,侍/產假,hour,hour,false,True,inner,entry
5,True,800,385e7625-a0fb-4337-8226-43a2f3eac4a2,,恩恤假,hour,day,false,False,inner,entry


In [25]:
df = vacation_types_df
# df = vacation_types_df.loc[vacation_types_df["leave_name"] == "年假(天)", "leave_code"]
# leave_code = df.iloc[0]
# opuserids: List[str] = opuserids_df.iloc[:30, 0].tolist()
# opusers_leave_records = get_opusers_leave_records(leave_code, opuserids)
df

,freedom_leave,hours_in_per_day,leave_code,leave_hour_ceil,leave_name,leave_time_ceil_min_unit,leave_view_unit,natural_day_leave,paid_leave,source,when_can_leave
0,True,800,f60377e7-9c60-48bb-91e2-5d02f4415361,up,年假(小時),halfHour,hour,false,False,inner,entry
1,True,800,c4d0316f-c234-4d89-8c66-c90b5931d072,,年假(天),NaN,day,false,False,inner,entry
2,True,800,c02aa600-8dce-4b63-9945-35d346464ef6,down,病假,halfHour,hour,false,False,inner,entry
3,True,800,a9047b4b-0d7f-4d7c-a57d-6cb89f7d3b91,,考試假,hour,day,false,False,inner,entry
4,True,800,52ed3423-0612-41c4-ace8-dc6b0c098030,down,侍/產假,hour,hour,false,True,inner,entry
5,True,800,385e7625-a0fb-4337-8226-43a2f3eac4a2,,恩恤假,hour,day,false,False,inner,entry
